In [234]:
import pandas as pd

In [235]:
df = pd.read_csv("spotify_millsongdata.csv")

In [236]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [237]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [238]:
df.shape

(57650, 4)

In [239]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [240]:
df = df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [241]:
df.head(10)

,artist,song,text
0,Kylie Minogue,On A Night Like This,Don't say it's like a fantasy \r\nWhen you kn...
1,Elton John,Cry To Heaven,I found a black beret on the street today \r\...
2,Hillsong United,Faithful To The End,Faithful to the End \r\n \r\nI'm saved from ...
3,Beauty And The Beast,Me,"Gaston \r\n \r\nYou've been dreaming, just o..."
4,Yeah Yeah Yeahs,Hysteric,No longer no longer \r\nWhat you ask \r\nStr...
5,Lynyrd Skynyrd,I Know A Little,"Yes sir \r\n \r\nWell the bigger the city, w..."
6,Billy Joel,The,"Well I'm on the Downeaster ""Alexa"" \r\nAnd I'..."
7,Vanilla Ice,Go Ill,[Verse 1] \r\nGet on up...i walk the beat and...
8,Stevie Wonder,Sensuous Whisper,"I've seen you, I saw you, I see you \r\nWitho..."
9,Peter Gabriel,Let It Be (Live),When I find myself in times of trouble \r\nMo...


In [242]:
df['text'][0]

"Don't say it's like a fantasy  \r\nWhen you know this is how it should be  \r\nYou kiss me  \r\nI'm falling can you hear me calling  \r\nYou touch me, I want you  \r\nFeels like I've always known you  \r\n  \r\nOn a night like this  \r\nI want to stay forever (stay forever)  \r\nOn a night like this  \r\nJust want to be together  \r\nOn a night like this  \r\n  \r\n(On a, on a, on, on a, on a, on)  \r\n(On a, on a, on, on a, on a, on)  \r\nNow I'm getting closer to you  \r\nHold me, I just can't be without you  \r\nYou kiss me, I'm falling  \r\nIt's your name I'm calling  \r\nYou touch me, I want you  \r\nFeels like I've always known you  \r\n  \r\nOn a night like this  \r\nI want to stay forever (stay forever)  \r\nOn a night like this  \r\nJust want to be together  \r\nOn a night like this  \r\n  \r\nSeems like I've known you a lifetime (lifetime)  \r\nNow it's time to make you mine  \r\nOn a night like this  \r\nOn a night like this, oh  \r\n  \r\n(You kiss me, I'm falling)  \r\n(Y

In [243]:
# df = df.sample(5000)

In [244]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [245]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', '', regex = True)

In [246]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [247]:
df['text'] = df['text'].apply(lambda x: tokenization(x))


In [248]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [249]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [250]:
similarity[0]

array([1.        , 0.03588944, 0.04415938, ..., 0.28394459, 0.01670204,
       0.09601833])

In [251]:
df[df['song'] == 'Crying Over You']


,artist,song,text


In [252]:

def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [253]:
recommendation('Crying Over You')


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))